In [1]:
# this part of the codes fetches requirements
import hashlib
from phe import paillier
import random as rd
# import numpy as np

In [2]:
#  this part of the code creates unique BID for all participants by the centeral authority
listOfIDS = []
numberOfBidders = 20
for i in range(0,numberOfBidders):
  i = str(i)
  listOfIDS.append(hashlib.sha256(i.encode('utf-8')).hexdigest())
i = 0

In [3]:
#this part creates the key pairs for all participants
dictionaryOfPublicKeys = {}
dictionaryOfPrivateKeys = {}
publicKeyAuc, privateKeyAuc = paillier.generate_paillier_keypair(n_length= 128)
for i in range(0, numberOfBidders):
    x,y = paillier.generate_paillier_keypair(n_length=256)
    dictionaryOfPublicKeys[listOfIDS[i]],dictionaryOfPrivateKeys[listOfIDS[i]]  = x ,y
i = 0

end of the Registration phase and start of Bid Submission Phase

In [4]:
# this part of the code generate random number and distinct bid values for all bidders and Auctioneer
dictionaryOfRandomNumbers = {}
dictionaryOfBids = {}
bids  = rd.sample(range(1, 100 ), numberOfBidders)
for i in range(0,numberOfBidders):
    r  = rd.randint(1,2000)
    dictionaryOfRandomNumbers[listOfIDS[i]] =  r
    dictionaryOfBids[listOfIDS[i]] = bids[i]
i = 0
print(dictionaryOfBids)
print(dictionaryOfRandomNumbers)


{'5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9': 58, '6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b': 36, 'd4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35': 94, '4e07408562bedb8b60ce05c1decfe3ad16b72230967de01f640b7e4729b49fce': 19, '4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328cb08b5531fcacdabf8a': 32, 'ef2d127de37b942baad06145e54b0c619a1f22327b2ebbcfbec78f5564afe39d': 15, 'e7f6c011776e8db7cd330b54174fd76f7d0216b612387a5ffcfb81e6f0919683': 74, '7902699be42c8a8e46fbbb4501726517e86b22c56a189f7625a6da49081b2451': 98, '2c624232cdd221771294dfbb310aca000a0df6ac8b66b696d90ef06fdefb64a3': 30, '19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebabe5ef03f7c3017bb5b7': 86, '4a44dc15364204a80fe80e9039455cc1608281820fe2b24f1e5233ade6af1dd5': 64, '4fc82b26aecb47d2868c4efbe3581732a3e7cbcc6c2efb32062c08170a05eeb8': 55, '6b51d431df5d7f141cbececcf79edf3dd861c3b4069f0b11661a3eefacbba918': 89, '3fdba35f04dc8c462986c992bcf875546257113072a909c162f7e470e581e2

In [5]:
# this PART OF THE CODE SIMULATES BIDDER GENERATING PRODUCT AND THAN SENDING TO NEXT BIDDER
finalSubmittedDataFirstPhase = [] #will be sused to stored the final resuklt of the two bidder for bid submission
for i in range(0,numberOfBidders):
    id = listOfIDS[i]
    dataPerBidder= []
    # print(dictionaryOfBids[id] * dictionaryOfRandomNumbers[id])
    encrytionOfBidAndRandomNumber = publicKeyAuc.raw_encrypt(dictionaryOfBids[id] + dictionaryOfRandomNumbers[id])
    # this part of code is forwarding the encrytionOfBidAndRandomNumber to all other bidders and after addition of the random number of the next bidder it is sent to the Auctioneer immediately
    for j in range(0,numberOfBidders):
        if j != i:
            idOfNextBidder = listOfIDS[j]
            pubKeyNextBidder, priKeyNextBidder= dictionaryOfPublicKeys[idOfNextBidder],dictionaryOfPrivateKeys[idOfNextBidder]
            e = pubKeyNextBidder.raw_encrypt(encrytionOfBidAndRandomNumber)
            e = priKeyNextBidder.raw_decrypt(e)
            encrytionOfBidAndBothRandomNumber = e * pow(publicKeyAuc.g,dictionaryOfRandomNumbers[idOfNextBidder])
            # this part of the code simulates the storing of the data of the bid summision to auctioneer data
            dataPerBidder.append(privateKeyAuc.raw_decrypt(encrytionOfBidAndBothRandomNumber))
        else:
            dataPerBidder.append(0)
    finalSubmittedDataFirstPhase.append(dataPerBidder)
i,j = 0,0

In [6]:
for i in range(len(finalSubmittedDataFirstPhase)):
    print(finalSubmittedDataFirstPhase[i])
i = 0

[0, 3466, 3160, 1847, 3104, 2732, 3259, 2081, 2129, 2088, 2161, 2943, 3069, 2624, 2336, 3659, 2345, 3413, 2264, 2265]
[3444, 0, 3008, 1695, 2952, 2580, 3107, 1929, 1977, 1936, 2009, 2791, 2917, 2472, 2184, 3507, 2193, 3261, 2112, 2113]
[3196, 3066, 0, 1447, 40481112857437914693511949302173694953, 2332, 2859, 1681, 1729, 1688, 1761, 2543, 2669, 2224, 1936, 3259, 1945, 3013, 1864, 1865]
[1808, 1678, 1372, 0, 76025464948198160777811654082277838625, 944, 1471, 293, 341, 300, 373, 1155, 1281, 836, 548, 1871, 557, 1625, 476, 477]
[3078, 2948, 2642, 1329, 0, 2214, 2741, 1563, 1611, 1570, 1643, 2425, 2551, 2106, 1818, 3141, 1827, 2895, 1746, 1747]
[2689, 2559, 2253, 940, 2197, 0, 2352, 1174, 1222, 1181, 1254, 2036, 2162, 1717, 1429, 2752, 1438, 2506, 1357, 1358]
[3275, 3145, 2839, 1526, 2783, 2411, 0, 1760, 1808, 1767, 1840, 2622, 2748, 2303, 2015, 3338, 2024, 3092, 1943, 1944]
[2121, 1991, 1685, 372, 1629, 1257, 1784, 0, 654, 613, 686, 1468, 1594, 1149, 861, 2184, 870, 1938, 789, 790]
[2101, 

implementation of the second phase here auctioneer reforward each packet to every bidder to multiply their random number.

In [7]:
#this works for a step of one
# for i in range(numberOfBidders):
#     sum = 0
#     for j in range(numberOfBidders):
#         if j + i < numberOfBidders:
#             sum = sum + finalSubmittedDataFirstPhase[j][j+i]
#         else:
#             sum = sum + finalSubmittedDataFirstPhase[j][(j+i)% numberOfBidders]
#     print(sum)   
# this works for any random step that has been taken in range (1,numberOfBidders) 
k = rd.randint(1,9)
print(k)
for i in range(numberOfBidders):
    sum = 0
    for j in range(numberOfBidders):
        if j + k < numberOfBidders:
            sum = sum + finalSubmittedDataFirstPhase[j][j+k]
        else:
            #we took the mod so that the next bidder value should come to the range from the number of the bidders
            sum = sum + finalSubmittedDataFirstPhase[j][(j+k)% numberOfBidders]
    print(sum)  

4
37088
37088
37088
37088
37088
37088
37088
37088
37088
37088
37088
37088
37088
37088
37088
37088
37088
37088
37088
37088


In [8]:
# this function chjoosews the best buyer based on the comparison it works as the auctrioneer is comparing each pair

highestbidder = 0 
def chooseBestBuyer():
    global highestbidder
    for i in range(0,numberOfBidders):
        highestbidder = i
        for j in range(0,numberOfBidders):
            if i != j:
                if finalSubmittedDataFirstPhase[i][j] < finalSubmittedDataFirstPhase[j][i]:
                    highestbidder  = j
                    i = j
            else:
                continue
                

#     global highestbidder
#     for i in range(0,numberOfBidders):   
#         for j in range(0,numberOfBidders):
#                 if i == j : 
#                      pass
#                 else:
#                      if (finalSubmittedDataFirstPhase[i][j])  > (finalSubmittedDataFirstPhase[j][i]):
#                          highestbidder = j
chooseBestBuyer()
print(highestbidder)
dictionaryOfBids

7


{'5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9': 58,
 '6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b': 36,
 'd4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35': 94,
 '4e07408562bedb8b60ce05c1decfe3ad16b72230967de01f640b7e4729b49fce': 19,
 '4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328cb08b5531fcacdabf8a': 32,
 'ef2d127de37b942baad06145e54b0c619a1f22327b2ebbcfbec78f5564afe39d': 15,
 'e7f6c011776e8db7cd330b54174fd76f7d0216b612387a5ffcfb81e6f0919683': 74,
 '7902699be42c8a8e46fbbb4501726517e86b22c56a189f7625a6da49081b2451': 98,
 '2c624232cdd221771294dfbb310aca000a0df6ac8b66b696d90ef06fdefb64a3': 30,
 '19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebabe5ef03f7c3017bb5b7': 86,
 '4a44dc15364204a80fe80e9039455cc1608281820fe2b24f1e5233ade6af1dd5': 64,
 '4fc82b26aecb47d2868c4efbe3581732a3e7cbcc6c2efb32062c08170a05eeb8': 55,
 '6b51d431df5d7f141cbececcf79edf3dd861c3b4069f0b11661a3eefacbba918': 89,
 '3fdba35f04dc8c462986c992bcf875546257113072a909c16

In [9]:
sum = 0
for i in range(numberOfBidders):
    id = listOfIDS[i]
    r = dictionaryOfRandomNumbers[id]
    sum = sum + r
sum
i = 0